<a href="https://colab.research.google.com/github/Hamza-benAmmar/Sudoku-game/blob/main/Sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time

class SudokuCSP:
    def __init__(self, puzzle):
        self.grid = puzzle

    def is_valid(self, row, col, num):
        # Check if 'num' is not in the same row
        if num in self.grid[row]:
            return False

        # Check if 'num' is not in the same column
        if num in [self.grid[i][col] for i in range(9)]:
            return False

        # Check if 'num' is not in the same 3x3 box
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if self.grid[i][j] == num:
                    return False
        return True

    def basic_find_empty_location(self):
        # Find an empty position (0) in the grid
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] == 0:
                    return i, j
        return None

    def count_remaining_values(self, row, col):
        # Count the number of remaining values for a given cell
        count = 0
        for num in range(1, 10):
            if self.is_valid(row, col, num):
                count += 1
        return count

    def find_empty_location_with_MRV_heuristic(self):
            # Find an empty position (0) in the grid with the MRV heuristic
            empty_positions = [(i, j, self.count_remaining_values(i, j)) for i in range(9) for j in range(9) if self.grid[i][j] == 0]
            empty_positions.sort(key=lambda x: x[2])  # Sort by the number of remaining values
            return empty_positions[0][:2] if empty_positions else None

    def count_constraints(self, row, col, num):
        # Count the number of constraints for a given value in an empty cell
        count = 0
        for i in range(9):
            if self.grid[row][i] == 0 and not self.is_valid(row, i, num):
                count += 1
            if self.grid[i][col] == 0 and not self.is_valid(i, col, num):
                count += 1

        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(start_row, start_row + 3):
            for j in range(start_col, start_col + 3):
                if self.grid[i][j] == 0 and not self.is_valid(i, j, num):
                    count += 1

        return count

    # Backtracking solver with LCV heuristic
    def solve_with_LCV(self):
        start_time = time.time()  # Start the timer

        # Find an empty position
        empty_location = self.find_empty_location_with_MRV_heuristic()

        # If there is no empty position, the puzzle is solved
        if not empty_location:
            elapsed_time = time.time() - start_time
            print(f"Sudoku solved in {elapsed_time:.6f} seconds")
            return True

        row, col = empty_location

        # Get possible values for the empty cell and sort them by least constraining values
        possible_values = [(num, self.count_constraints(row, col, num)) for num in range(1, 10) if self.is_valid(row, col, num)]
        possible_values.sort(key=lambda x: x[1])

        # Try placing a number from least constraining to most constraining
        for num, _ in possible_values:
            # If the placement is valid, update the grid
            self.grid[row][col] = num

            # Recursively try to solve the rest of the puzzle
            if self.solve_with_LCV():
                return True

            # If placing the current number doesn't lead to a solution, backtrack
        self.grid[row][col] = 0

        # If no number leads to a solution, backtrack
        elapsed_time = time.time() - start_time
        print(f"No solution found. Elapsed time: {elapsed_time:.6f} seconds")
        return False

    def basic_solve(self):
        start_time = time.time()
        # Find an empty position
        empty_location = self.basic_find_empty_location()

        # If there is no empty position, the puzzle is solved
        if not empty_location:
            elapsed_time = time.time() - start_time
            print(f"Sudoku solved in {elapsed_time:.6f} seconds")
            return True

        row, col = empty_location

        # Try placing a number from 1 to 9
        for num in range(1, 10):
            if self.is_valid(row, col, num):
                # If the placement is valid, update the grid
                self.grid[row][col] = num

                # Recursively try to solve the rest of the puzzle
                if self.basic_solve():
                    return True

                # If placing the current number doesn't lead to a solution, backtrack
                self.grid[row][col] = 0

        # If no number leads to a solution, backtrack
        return False

    def solve_with_MRV(self):
        start_time = time.time()
        # Find an empty position
        empty_location = self.basic_find_empty_location()

        # If there is no empty position, the puzzle is solved
        if not empty_location:
            elapsed_time = time.time() - start_time
            print(f"Sudoku solved in {elapsed_time:.6f} seconds")
            return True

        row, col = empty_location

        # Try placing a number from 1 to 9
        for num in range(1, 10):
            if self.is_valid(row, col, num):
                # If the placement is valid, update the grid
                self.grid[row][col] = num

                # Recursively try to solve the rest of the puzzle
                if self.solve_with_MR():
                    return True

                # If placing the current number doesn't lead to a solution, backtrack
                self.grid[row][col] = 0

        # If no number leads to a solution, backtrack
        return False

    def display(self):
        # Display the solved Sudoku grid
        for row in self.grid:
            print(" ".join(map(str, row)))


# Example Usage:
initial_state = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

sudoku_csp = SudokuCSP(initial_state)

if sudoku_csp.basic_solve():
    print("Sudoku solved:")
    sudoku_csp.display()
else:
    print("No solution exists with backtracking.")

if sudoku_csp.solve_with_MRV():
    print("Sudoku solved with MRV:")
    sudoku_csp.display()
else:
    print("No solution exists with MRV.")

if sudoku_csp.solve_with_LCV():
    print("Sudoku solved with LCV:")
    sudoku_csp.display()

else:
    print("No solution exists with LCV.")

Sudoku solved in 0.000009 seconds
Sudoku solved:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
Sudoku solved in 0.000020 seconds
Sudoku solved with MRV:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
Sudoku solved in 0.000016 seconds
Sudoku solved with LCV:
5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
